In this notebook we use the `./final/dataset.npz` dataset to train multiple ML models to create a surrogate models. The comparison between models will be presented in the next notebook.

During training process dataset is split into train, test and validate sets of sizes `700000`, `150000` and `150000`.

List of tested methods (with links to used implementations):
1. [Neural Networks](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html)
2. Linear regression model:
    1. [Elastic Net](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html#sklearn.linear_model.ElasticNet) L1 and L2 regularization combined
3. Decistion Trees models:
    1. [Boosting](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.HistGradientBoostingRegressor.html)
    2. [Bagging](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingRegressor.html)
    3. [Random Forrest](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html)
    4. [Extra Trees](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.ExtraTreesRegressor.html)
4. [k-nearest neighbors](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html)

For each ML method we use [Optuna](https://optuna.readthedocs.io) to find best performing set of parameters.

In [19]:
RUN_ITERATIONS_PERCENT = 100  # runs only X percent of iterations. pick small number to test if notebook is working fine

In [2]:
# Loading dataset
import numpy as np
import os
from pathlib import Path

Path("../final/ml_models").mkdir(parents=True, exist_ok=True)

input_and_output = np.load("../final/dataset.npz")
inputs  = input_and_output["inputs"].astype(np.float64)
outputs = input_and_output["outputs"].astype(np.float64)

print(f"inputs shape: {inputs.shape} dtype: {inputs.dtype}, outputs shape: {outputs.shape}, dtype: {outputs.dtype}")
dataset_size = inputs.shape[0]

inputs shape: (1000000, 11) dtype: float64, outputs shape: (1000000, 200), dtype: float64


In [3]:
# In this problem we are interrested in order of magnitude rather than absolute value of the tumour size.
# To train the models output is first transformed with log_10. When the tumour size is smaller than 10^-9 L, 
# there is no way to find it, so we can limit the lower bound of tumour size with 10^-9

LOWER_LIMIT = -7

def output_transform(outputs: np.array) -> np.array:
    x = np.copy(outputs)
    zeros_in_output = x <= 0
    x[zeros_in_output] = 1
    y = np.log10(x)
    y[zeros_in_output] = LOWER_LIMIT
    y[y < LOWER_LIMIT] = LOWER_LIMIT
    return y
    
def output_untransform(transformed_outputs: np.array) -> np.array:
    lower_limits = transformed_outputs <= LOWER_LIMIT
    z = 10 ** transformed_outputs
    z[lower_limits] = 0
    return z

test_output = np.array([10**(-8), 10**(-5), 10**(-10), 10**(-9),1,2,10,0.0])

print(f"test output {test_output}")
print(f"transformed output: {output_transform(test_output)}")
print(f"original output is untouched after transform: {test_output}")
print(f"transformed and untransformed output: {output_untransform(output_transform(test_output))}")

outputs_order_of_magnitude = output_transform(outputs)

test output [1.e-08 1.e-05 1.e-10 1.e-09 1.e+00 2.e+00 1.e+01 0.e+00]
transformed output: [-7.      -5.      -7.      -7.       0.       0.30103  1.      -7.     ]
original output is untouched after transform: [1.e-08 1.e-05 1.e-10 1.e-09 1.e+00 2.e+00 1.e+01 0.e+00]
transformed and untransformed output: [0.e+00 1.e-05 0.e+00 0.e+00 1.e+00 2.e+00 1.e+01 0.e+00]


In [4]:
# Input data has an extra column with all ones - that we get rid of before training the model

def drop_treatment(input_data: np.ndarray) -> np.ndarray:
    """Drops treatment data from the dataset"""
    if input_data.shape[1] == 11:
        return input_data[:, 1:]

    return input_data

input_without_treatment = drop_treatment(inputs)

In [5]:
# Splitting dataset into train, test, validate subsets
train_size = int(dataset_size * 0.7)
test_size = int(dataset_size * 0.15)

X_train = input_without_treatment[:train_size, :]
Y_train = outputs_order_of_magnitude[:train_size, :]
print(f"train sizes: {X_train.shape}, {Y_train.shape}")
X_test = input_without_treatment[train_size:(train_size + test_size), :]
Y_test = outputs_order_of_magnitude[train_size:(train_size + test_size), :]
print(f"test sizes: {X_test.shape}, {Y_test.shape}")

train sizes: (700000, 10), (700000, 200)
test sizes: (150000, 10), (150000, 200)


In [6]:
# scaling inputs
from sklearn.preprocessing import MinMaxScaler
import pickle
from pathlib import Path


LOGNORMAL_PARAMETERS = (1, 2)

class CustomScaler:
    def __init__(self):
        super().__init__()
        self.scaler = MinMaxScaler()
        self.plot_loval = [0.0] * len(LOGNORMAL_PARAMETERS)
        self.plot_hival = [1.0] * len(LOGNORMAL_PARAMETERS)

    def transform(self, x: np.ndarray, copy=None) -> np.ndarray:
        res = self.scaler.transform(x)
        for i, parameter_index in enumerate(LOGNORMAL_PARAMETERS):
            res[:, parameter_index] = (x[:, parameter_index] - self.plot_loval[i]) / (self.plot_hival[i] - self.plot_loval[i])

        return res

    def fit(self, x, copy=None):
        self.scaler.fit(x)
        for i, parameter_index in enumerate(LOGNORMAL_PARAMETERS):
            column_values = x[:, parameter_index]

            quantile_1, quantile_3 = np.quantile(column_values, [0.25, 0.75], axis=0)
            iqr = quantile_3 - quantile_1

            loval = quantile_1 - 1.5 * iqr
            hival = quantile_3 + 1.5 * iqr

            wiskhi = np.compress(column_values <= hival, column_values)
            wisklo = np.compress(column_values >= loval, column_values)
            actual_hival = np.max(wiskhi)
            actual_loval = np.min(wisklo)

            self.plot_loval[i] = actual_loval
            self.plot_hival[i] = actual_hival

        return self

    def inverse_transform(self, x, copy=None):
        res = self.scaler.inverse_transform(x)
        for i, parameter_index in enumerate(LOGNORMAL_PARAMETERS):
            res[:, parameter_index] = x[:, parameter_index] * (self.plot_hival[i] - self.plot_loval[i]) + self.plot_loval[i]
        return res

scaler_path = Path(f"../final/scaler.pickle")
scaler = None
if scaler_path.exists():
    with scaler_path.open("rb") as scaler_file:
        scaler = pickle.load(scaler_file)
else:
    scaler = CustomScaler().fit(X_train)
    with scaler_path.open("wb") as opened_file:
        pickle.dump(scaler, opened_file)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [7]:
# applying pca to outputs
from sklearn.decomposition import PCA


PCA_COMPONENTS=16

pca_path = Path(f"../final/pca{PCA_COMPONENTS}.pickle")

if pca_path.exists():
    with pca_path.open("rb") as opened_file:
        pca = pickle.load(opened_file)
    Y_train_pca = pca.transform(Y_train)
else: 
    pca = PCA(n_components=PCA_COMPONENTS)
    Y_train_pca = pca.fit_transform(Y_train)
    with pca_path.open("wb") as opened_file:
        pickle.dump(pca, opened_file)

print(pca.explained_variance_ratio_)
from functools import reduce
print(list(reduce(lambda a, b: a + [a[-1] - b], pca.explained_variance_ratio_, [1.0]))[1:])

[9.65896156e-01 3.09969465e-02 2.13722285e-03 6.93076643e-04
 1.63051574e-04 5.64522465e-05 2.52438747e-05 1.34355642e-05
 6.67212660e-06 3.98353564e-06 2.43601939e-06 1.58933641e-06
 1.01770830e-06 6.80668693e-07 4.71992976e-07 3.33020978e-07]
[0.03410384435084102, 0.003106897801798597, 0.0009696749484840702, 0.00027659830525620223, 0.00011354673170347712, 5.7094485160550534e-05, 3.185061047702586e-05, 1.8415046228782943e-05, 1.1742919633474705e-05, 7.759383996452503e-06, 5.323364604422705e-06, 3.7340281921318385e-06, 2.71631988942079e-06, 2.0356511961545228e-06, 1.5636582199018604e-06, 1.230637241455825e-06]


In [8]:
# Example of limitting running time of the loop by scheduling an sigalrm and adding a handler for it.

import optuna
import time
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
import psutil
import os
import signal

class TimeoutError(Exception):
    pass

def timeout_handler(*args):
    raise TimeoutError(f"timeout {args}")

def keyboard_interrupt_handler(*args):
    os.kill(os.getpid(), signal.SIGINT)
    
signal.signal(signal.SIGALRM, timeout_handler)

try:
    signal.alarm(2)
    time.sleep(1)
    signal.alarm(0)
    print(f"1. no exception")
except TimeoutError as e:
    print(f"1. message: {e}") 
except KeyboardInterrupt as e:
    print(f"1. KEYBOARD: {e}") 

try:
    signal.alarm(2)
    time.sleep(5)
    signal.alarm(0)
    print(f"2. no exception")
except TimeoutError as e:
    print(f"2. message: {e}")
except KeyboardInterrupt as e:
    print(f"2. keyboard: {e}") 

1. no exception
2. message: timeout (14, <frame at 0x12fb1d010, file '/var/folders/jg/gsbj6mcn3ldd9z2krrny7lx80000gq/T/ipykernel_18237/3465348698.py', line 35, code <module>>)


In [9]:
def save_best_model(model, error, trial):
    try: 
        best_value = trial.study.best_value
    except: 
        best_value = float('inf')
    if error < best_value:
        print(f"Updating {trial.study.study_name}.pickle")
        with Path(f"../final/ml_models/{trial.study.study_name}.pickle").open("wb") as opened_file:
            pickle.dump(model, opened_file)
            
def mse_on_test_dataset(model):
    global X_test_scaled, pca, Y_test
    
    Y_predict_pca = model.predict(X_test_scaled)
    Y_predict = pca.inverse_transform(Y_predict_pca)
    return mean_squared_error(Y_test, Y_predict)

In [15]:
# Hyperparameter optimization for neural network

from sklearn.neural_network import MLPRegressor

common_params={
    "tol": 3e-6,
    "n_iter_no_change": 5,
    "random_state": 42,
    "warm_start": False,
    "batch_size": 10000,
    "max_iter": 5000
}

def objective(trial):
    global common_params
    
    phase = min((trial.number // 30), 3)
    training_sizes = (0.05, 0.1, 0.3, 1.0)
    training_max_duration_s = (300, 600, 1200, 3600)
    training_size = int(training_sizes[phase] * X_train_scaled.shape[0])
    max_duration_s = training_max_duration_s[phase]

    # learning_rate = trial.suggest_categorical("learning_rate", ["constant", "invscaling", "adaptive"])
    learning_rate = "constant"
    model_params = {
        **common_params,
        "alpha": trial.suggest_float("alpha", 0.001, 1.0, log=True),
        "learning_rate": learning_rate,
        "learning_rate_init": trial.suggest_float("learning_rate_init", 0.0001, 0.1, log=True),
        "power_t": trial.suggest_float("power_t", 0.1, 2.0, log=True) if learning_rate == "invscaling" else 0.5,
        "hidden_layer_sizes": [
            trial.suggest_int(f"layer1/3", 600, 1200, step=200),
            trial.suggest_int(f"layer2/3", 200, 600, step=50),
            trial.suggest_int(f"layer3/3", 10, 30, step=5),
        ],
        "random_state": trial.suggest_categorical("random_state", list(range(0, 32))),
#         "hidden_layer_sizes": [700, 200, 30]
    }
    
    trial.set_user_attr("training_size", training_size)
    trial.set_user_attr("max_duration_s", max_duration_s)
    trial.set_user_attr("model_params", model_params)
    
    model = MLPRegressor(**trial.user_attrs["model_params"])
    
    try:
        signal.alarm(max_duration_s)
        with threadpool_limits(limits=8, user_api='blas'):
            model.fit(X_train_scaled[:training_size,:], Y_train_pca[:training_size,:])
        signal.alarm(0)
        error = mse_on_test_dataset(model)
        save_best_model(model, error, trial)
            
    except TimeoutError:
        raise optuna.exceptions.TrialPruned()
    return error

study = optuna.create_study(study_name="MLPRegressor", storage='sqlite:///../final/optuna.db', load_if_exists=True)
trials_due = 110 * RUN_ITERATIONS_PERCENT // 100 - len(study.trials)
if trials_due > 0:
    previous_handler = signal.signal(signal.SIGALRM, keyboard_interrupt_handler)
    study.optimize(objective, n_trials=trials_due)
    signal.signal(signal.SIGALRM, previous_handler)
    
print(f"model: {study.study_name} test dataset error: {study.best_value} best_params: {study.best_params}")

[I 2023-03-24 07:12:41,627] Using an existing study with name 'MLPRegressor' instead of creating a new one.
[I 2023-03-24 07:48:23,230] Trial 100 finished with value: 0.00029294497816456243 and parameters: {'alpha': 0.002888202482792453, 'learning_rate_init': 0.0011042917115084308, 'layer1/3': 1000, 'layer2/3': 450, 'layer3/3': 15, 'random_state': 8}. Best is trial 95 with value: 0.0002649237647044346.
[I 2023-03-24 08:22:02,435] Trial 101 finished with value: 0.00030768758303624364 and parameters: {'alpha': 0.003152614359252359, 'learning_rate_init': 0.00112181688145657, 'layer1/3': 1000, 'layer2/3': 450, 'layer3/3': 15, 'random_state': 8}. Best is trial 95 with value: 0.0002649237647044346.
[I 2023-03-24 09:02:48,920] Trial 102 finished with value: 0.0008420089919682261 and parameters: {'alpha': 0.0029003074270391466, 'learning_rate_init': 0.001385247560864844, 'layer1/3': 1000, 'layer2/3': 450, 'layer3/3': 15, 'random_state': 8}. Best is trial 95 with value: 0.0002649237647044346.
[

model: MLPRegressor test dataset error: 0.0002649237647044346 best_params: {'alpha': 0.0036189832082902966, 'layer1/3': 1000, 'layer2/3': 400, 'layer3/3': 20, 'learning_rate_init': 0.0007411079532929595, 'random_state': 8}


In [20]:
# Test how neural network perform with 10 minute training cap on full dataset
from sklearn.neural_network import MLPRegressor
from threadpoolctl import threadpool_limits

for learning_rate in ["constant", "invscaling", "adaptive"]:
    max_duration_s = 300
    common_params={
        "tol": 1e-5,
        "n_iter_no_change": 5,
        "random_state": 42,
        "warm_start": False,
        "max_iter": 2000,
        "batch_size": 10000,
        "learning_rate": learning_rate
    }

    def objective(trial):
        global common_params
        global max_duration_s

        alpha = trial.suggest_float("alpha", 0.0001, 1.0, log=True)
        learning_rate_init = trial.suggest_float("learning_rate_init", 0.0001, 0.05, log=True)
        power_t = trial.suggest_float("power_t", 0.1, 2.0, log=True) if learning_rate == "invscaling" else 0.5

        model_params = {
            **common_params,
            "alpha": alpha,
            "learning_rate_init": learning_rate_init,
            "power_t": power_t,
            "hidden_layer_sizes": [
                trial.suggest_int(f"layer1/3", 600, 1200, step=200),
                trial.suggest_int(f"layer2/3", 200, 600, step=50),
                trial.suggest_int(f"layer3/3", 10, 50, step=5)
            ],
            "batch_size": trial.suggest_int("batch_size", 10000, 25000, step=5000),
            "random_state": trial.suggest_categorical("random_state", list(range(0, 32))),
            #"hidden_layer_sizes": [600, 200, 50]
        }
        print(model_params)

        trial.set_user_attr("model_params", model_params)

        model = MLPRegressor(**trial.user_attrs["model_params"])
        trial.set_user_attr("training_size", 1.0)
        trial.set_user_attr("max_duration_s", max_duration_s)

        try:
            signal.alarm(max_duration_s)
            with threadpool_limits(limits=8, user_api='blas'):
                model.fit(X_train_scaled, Y_train_pca)
            signal.alarm(0)
            error = mse_on_test_dataset(model)
            save_best_model(model, error, trial)
            
        except TimeoutError:
            raise optuna.exceptions.TrialPruned()
        return error

    study = optuna.create_study(study_name=f"MLPRegressor_{learning_rate}_{max_duration_s}", storage='sqlite:///../final/optuna.db', load_if_exists=True)
    trials_due = 40 * RUN_ITERATIONS_PERCENT // 100 - len(study.trials)
    if trials_due > 0:
        previous_handler = signal.signal(signal.SIGALRM, keyboard_interrupt_handler)
        study.optimize(objective, n_trials=trials_due)
        signal.signal(signal.SIGALRM, previous_handler)

    print(f"model: {study.study_name} test dataset error: {study.best_value} best_params: {study.best_params}")

[I 2023-03-25 19:23:41,791] Using an existing study with name 'MLPRegressor_constant_300' instead of creating a new one.


{'tol': 1e-05, 'n_iter_no_change': 5, 'random_state': 11, 'warm_start': False, 'max_iter': 2000, 'batch_size': 20000, 'learning_rate': 'constant', 'alpha': 0.00046004556346854636, 'learning_rate_init': 0.00044155528151566787, 'power_t': 0.5, 'hidden_layer_sizes': [800, 400, 45]}


/Users/tpawlowski/Documents/qsp_emulation/venv/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")
[I 2023-03-25 19:28:45,031] Trial 4 finished with value: 0.011399999991353642 and parameters: {'alpha': 0.00046004556346854636, 'learning_rate_init': 0.00044155528151566787, 'layer1/3': 800, 'layer2/3': 400, 'layer3/3': 45, 'batch_size': 20000, 'random_state': 11}. Best is trial 2 with value: 0.0044855957224417264.


{'tol': 1e-05, 'n_iter_no_change': 5, 'random_state': 25, 'warm_start': False, 'max_iter': 2000, 'batch_size': 15000, 'learning_rate': 'constant', 'alpha': 0.00012821943809007743, 'learning_rate_init': 0.0005315209392756725, 'power_t': 0.5, 'hidden_layer_sizes': [600, 550, 25]}


/Users/tpawlowski/Documents/qsp_emulation/venv/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")
[I 2023-03-25 19:33:48,272] Trial 5 finished with value: 0.009478221636772472 and parameters: {'alpha': 0.00012821943809007743, 'learning_rate_init': 0.0005315209392756725, 'layer1/3': 600, 'layer2/3': 550, 'layer3/3': 25, 'batch_size': 15000, 'random_state': 25}. Best is trial 2 with value: 0.0044855957224417264.


...

{'tol': 1e-05, 'n_iter_no_change': 5, 'random_state': 14, 'warm_start': False, 'max_iter': 2000, 'batch_size': 15000, 'learning_rate': 'adaptive', 'alpha': 0.279220458627532, 'learning_rate_init': 0.004874277785323703, 'power_t': 0.5, 'hidden_layer_sizes': [1200, 250, 25]}


/Users/tpawlowski/Documents/qsp_emulation/venv/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")
[I 2023-03-26 05:22:30,738] Trial 38 finished with value: 0.010539428679915992 and parameters: {'alpha': 0.279220458627532, 'learning_rate_init': 0.004874277785323703, 'layer1/3': 1200, 'layer2/3': 250, 'layer3/3': 25, 'batch_size': 15000, 'random_state': 14}. Best is trial 28 with value: 0.0005397243537644719.


{'tol': 1e-05, 'n_iter_no_change': 5, 'random_state': 1, 'warm_start': False, 'max_iter': 2000, 'batch_size': 10000, 'learning_rate': 'adaptive', 'alpha': 0.10448540546457212, 'learning_rate_init': 0.012998024353688927, 'power_t': 0.5, 'hidden_layer_sizes': [600, 300, 15]}


[I 2023-03-26 05:27:01,616] Trial 39 finished with value: 0.00173551231131575 and parameters: {'alpha': 0.10448540546457212, 'learning_rate_init': 0.012998024353688927, 'layer1/3': 600, 'layer2/3': 300, 'layer3/3': 15, 'batch_size': 10000, 'random_state': 1}. Best is trial 28 with value: 0.0005397243537644719.


model: MLPRegressor_adaptive_300 test dataset error: 0.0005397243537644719 best_params: {'alpha': 0.06697506164069783, 'batch_size': 10000, 'layer1/3': 600, 'layer2/3': 300, 'layer3/3': 30, 'learning_rate_init': 0.010266982855378118, 'random_state': 20}


In [16]:
optuna.delete_study(study_name="MLPRegressor_constant_300", storage='sqlite:///../final/optuna.db')

KeyError: 'Record does not exist.'

In [18]:
# Linear regression with combined L1 and L2 priors as regularizer
from sklearn.linear_model import ElasticNet

common_params = {
    "random_state": 42,
    "tol": 1e-5
}

MAX_POLYNOMIAL_DEGREE = 3

def objective(trial):
    global common_params
    phase = min((trial.number // 30), 4)

    training_sizes = (0.03, 0.1, 0.3, 1.0)
    training_max_duration_s = (240, 360, 600, 1200)
    
    training_size = int(training_sizes[phase] * X_train_scaled.shape[0])
    max_duration_s = training_max_duration_s[phase]
    max_iter = [20000, 40000, 80000, 160000][phase]
    
    trial.set_user_attr("training_size", training_size)
    trial.set_user_attr("max_duration_s", max_duration_s)
    trial.set_user_attr("model_params", {
        **common_params,
        "max_iter": max_iter,
        "l1_ratio": trial.suggest_float("l1_ratio", 0.0, 1.0),
        "alpha": trial.suggest_float("alpha", 0.00001, 1.0, log=True),
        "fit_intercept": trial.suggest_categorical("fit_intercept", [True, False])
    })
    
    trial.set_user_attr("polynomial degree", trial.suggest_int("polynomial degree", 1, MAX_POLYNOMIAL_DEGREE))
    
    model = make_pipeline(
        PolynomialFeatures(trial.user_attrs["polynomial degree"]),
        ElasticNet(**trial.user_attrs["model_params"])
    )
    
    try:
        signal.alarm(max_duration_s)
        model.fit(X_train_scaled[:training_size,:], Y_train_pca[:training_size,:])
        signal.alarm(0)
        error = mse_on_test_dataset(model)
        save_best_model(model, error, trial)
    except TimeoutError:
        raise optuna.exceptions.TrialPruned()
    return error

study = optuna.create_study(study_name="ElasticNet", storage='sqlite:///../final/optuna.db', load_if_exists=True)
trials_due = 100 * RUN_ITERATIONS_PERCENT // 100 - len(study.trials)
if trials_due > 0:
    study.optimize(objective, n_trials=trials_due)
    
print(f"model: {study.study_name} test dataset error: {study.best_value} best_params: {study.best_params}")

[I 2023-03-21 15:15:05,958] A new study created in RDB with name: ElasticNet


Updating ElasticNet.pickle


[I 2023-03-21 15:15:11,563] Trial 0 finished with value: 0.10261961943221654 and parameters: {'l1_ratio': 0.2125353467369947, 'alpha': 0.0004221486333249773, 'fit_intercept': True, 'polynomial degree': 2}. Best is trial 0 with value: 0.10261961943221654.
[I 2023-03-21 15:17:08,667] Trial 1 finished with value: 0.10336975420999586 and parameters: {'l1_ratio': 0.11253674029614558, 'alpha': 0.0005185109299269791, 'fit_intercept': False, 'polynomial degree': 2}. Best is trial 0 with value: 0.10261961943221654.
/home/tpawlowski/qsp_emulation/venv/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.219e+02, tolerance: 3.019e+01
  model = cd_fast.enet_coordinate_descent(
[I 2023-03-21 15:22:11,386] Trial 2 pruned. 
/home/tpawlowski/qsp_emulation/venv/lib/python3.8/site-packages/sklear

model: ElasticNet test dataset error: 0.10261961943221654 best_params: {'alpha': 0.0004221486333249773, 'fit_intercept': True, 'l1_ratio': 0.2125353467369947, 'polynomial degree': 2}


In [20]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import HistGradientBoostingRegressor

def objective(trial):
    trial_params = {}
    # poisson requires y > 0 which is not true in this case
    trial_params["loss"] = trial.suggest_categorical("loss", ["squared_error", "absolute_error", "quantile"])
    
    if trial_params["loss"] == "quantile":
        trial_params["quantile"] = trial.suggest_float("quantile", 0, 1)
    
    trial_params["learning_rate"] = trial.suggest_float("learning_rate", 0.00001, 0.5, log=True)
    trial_params["max_iter"] = trial.suggest_int("max_iter", 20, 200, step=20)
    if trial.suggest_categorical("regularize", [True, False]):
        trial_params["l2_regularization"] = trial.suggest_float("l2_regularization", 0.00001, 1.0, log=True)
    
    trial.set_user_attr("model_params", {
        "random_state": 42,
        **trial_params
    })
    
    model = MultiOutputRegressor(HistGradientBoostingRegressor(**trial.user_attrs["model_params"]))
    
    phase = min((trial.number // 30), 4)
    
    training_sizes = (0.03, 0.1, 0.3, 1.0)
    training_max_duration_s = (240, 360, 600, 1200)
    
    training_size = int(training_sizes[phase] * X_train_scaled.shape[0])
    max_duration_s = training_max_duration_s[phase]
    
    trial.set_user_attr("training_size", training_size)
    trial.set_user_attr("max_duration_s", max_duration_s)
    
    try:
        signal.alarm(max_duration_s)
        model.fit(X_train_scaled[:training_size,:], Y_train_pca[:training_size,:])
        signal.alarm(0)
        error = mse_on_test_dataset(model)
        save_best_model(model, error, trial)
    except KeyboardInterrupt as e:
        raise e
    except Exception as e:
        print(e)
        raise optuna.exceptions.TrialPruned()
    return error

study = optuna.create_study(study_name="HistGradientBoostingRegressor", storage='sqlite:///../final/optuna.db', load_if_exists=True)
trials_due = 100 * RUN_ITERATIONS_PERCENT // 100 - len(study.trials)
if trials_due > 0:
    study.optimize(objective, n_trials=trials_due)

print(f"model: {study.study_name} test dataset error: {study.best_value} best_params: {study.best_params}")

[I 2023-03-21 15:38:39,525] Using an existing study with name 'HistGradientBoostingRegressor' instead of creating a new one.


model: HistGradientBoostingRegressor test dataset error: 0.014631947216599755 best_params: {'l2_regularization': 0.20213714120841555, 'learning_rate': 0.13215099864424212, 'loss': 'squared_error', 'max_iter': 140, 'regularize': True}


In [23]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import BaggingRegressor

def objective(trial):
    phase = min((trial.number // 30), 4)
    
    training_sizes = (0.1, 0.1, 0.3, 1.0)
    training_max_duration_s = (240, 360, 600, 1200)
    
    training_size = int(training_sizes[phase] * X_train_scaled.shape[0])
    max_duration_s = training_max_duration_s[phase]
    
    trial_params = {}
    
    trial_params["n_estimators"] = trial.suggest_int("max_iter", 10, 110, step=20)
    trial_params["max_samples"] = trial.suggest_int("max_samples", 10000, (training_size // 10000) * 10000, step=10000)
    trial_params["max_features"] = trial.suggest_int("max_features", 1, X_train_scaled.shape[1])
    trial_params["bootstrap"] = trial.suggest_categorical("bootstrap", [True, False])
    if trial_params["bootstrap"]:
        trial_params["oob_score"] = trial.suggest_categorical("oob_score", [True, False])
    trial_params["bootstrap_features"] = trial.suggest_categorical("bootstrap_features", [True, False])
    
    trial.set_user_attr("training_size", training_size)
    trial.set_user_attr("max_duration_s", max_duration_s)
    trial.set_user_attr("model_params", {
        "n_jobs": -1,
        "random_state": 42,
        **trial_params
    })
    
    model = MultiOutputRegressor(BaggingRegressor(**trial.user_attrs["model_params"]))
    
    try:
        signal.alarm(max_duration_s)
        model.fit(X_train_scaled[:training_size,:], Y_train_pca[:training_size,:])
        signal.alarm(0)
        error = mse_on_test_dataset(model)
        save_best_model(model, error, trial)
    except KeyboardInterrupt as e:
        raise e
    except Exception as e:
        print(e)
        raise optuna.exceptions.TrialPruned()
    return error

# try:
#     optuna.delete_study(study_name="BaggingRegressor", storage='sqlite:///../final/optuna.db')
# except:
#     pass

study = optuna.create_study(study_name="BaggingRegressor", storage='sqlite:///../final/optuna.db', load_if_exists=True)
trials_due = 100 * RUN_ITERATIONS_PERCENT // 100 - len(study.trials)
if trials_due > 0:
    study.optimize(objective, n_trials=trials_due)
    
print(f"model: {study.study_name} test dataset error: {study.best_value} best_params: {study.best_params}")

[I 2023-03-21 15:52:21,183] A new study created in RDB with name: BaggingRegressor


Updating BaggingRegressor.pickle


[I 2023-03-21 15:53:02,898] Trial 0 finished with value: 0.0264062521618683 and parameters: {'max_iter': 30, 'max_samples': 20000, 'max_features': 10, 'bootstrap': False, 'bootstrap_features': False}. Best is trial 0 with value: 0.0264062521618683.
[I 2023-03-21 15:55:00,712] Trial 1 finished with value: 0.3143542015744639 and parameters: {'max_iter': 90, 'max_samples': 70000, 'max_features': 6, 'bootstrap': False, 'bootstrap_features': True}. Best is trial 0 with value: 0.0264062521618683.
[I 2023-03-21 15:55:18,140] Trial 2 finished with value: 0.19056331769400767 and parameters: {'max_iter': 10, 'max_samples': 10000, 'max_features': 9, 'bootstrap': True, 'oob_score': True, 'bootstrap_features': True}. Best is trial 0 with value: 0.0264062521618683.
[I 2023-03-21 15:57:29,737] Trial 3 finished with value: 0.2211025704912809 and parameters: {'max_iter': 90, 'max_samples': 60000, 'max_features': 8, 'bootstrap': False, 'bootstrap_features': True}. Best is trial 0 with value: 0.026406252

model: BaggingRegressor test dataset error: 0.0264062521618683 best_params: {'bootstrap': False, 'bootstrap_features': False, 'max_features': 10, 'max_iter': 30, 'max_samples': 20000}


In [21]:
from sklearn.ensemble import RandomForestRegressor

def objective(trial):
    phase = min((trial.number // 30), 4)

    training_sizes = (0.01, 0.03, 0.1, 0.3, 1.0)
    training_max_duration_s = (120, 180, 360, 600, 1200)
    training_size = int(training_sizes[phase] * X_train_scaled.shape[0])
    max_duration_s = training_max_duration_s[phase]
    
    trial.set_user_attr("training_size", training_size)
    trial.set_user_attr("max_duration_s", max_duration_s)
    trial.set_user_attr("model_params", {
      "n_estimators": trial.suggest_int("n_estimators", 10, 200, step=10),
      "criterion": trial.suggest_categorical("criterion", ["squared_error", "absolute_error", "friedman_mse"]),
      "min_samples_split": trial.suggest_int("min_samples_split", 2, 5),
      "bootstrap": trial.suggest_categorical("bootstrap", [True]),
      "max_samples": trial.suggest_float("max_samples", 0.0, 0.2),
      "max_features": trial.suggest_int("max_features", 1, X_train_scaled.shape[1] // 2),
      "n_jobs": -1,
      "random_state": 42
    })
    
    model = RandomForestRegressor(**trial.user_attrs["model_params"])
    
    try:
        signal.alarm(max_duration_s)
        model.fit(X_train_scaled[:training_size,:], Y_train_pca[:training_size,:])
        Y_predict_pca = model.predict(X_test_scaled)
        Y_predict = pca.inverse_transform(Y_predict_pca)
        error = mean_squared_error(Y_test, Y_predict)
        signal.alarm(0)
    except KeyboardInterrupt as e:
        raise e
    except:
        raise optuna.exceptions.TrialPruned()
    return error

study = optuna.create_study(study_name="RandomForrest", storage='sqlite:///../final/optuna.db', load_if_exists=True)
trials_due = 100 * RUN_ITERATIONS_PERCENT // 100 - len(study.trials)
if trials_due > 0:
    study.optimize(objective, n_trials=trials_due)

print(f"model: {study.study_name} test dataset error: {study.best_value} best_params: {study.best_params}")

[I 2023-03-21 15:38:45,101] A new study created in RDB with name: RandomForrest
[I 2023-03-21 15:38:47,416] Trial 0 finished with value: 0.08256820963993508 and parameters: {'n_estimators': 110, 'criterion': 'friedman_mse', 'min_samples_split': 5, 'bootstrap': True, 'max_samples': 0.16416466786708495, 'max_features': 4}. Best is trial 0 with value: 0.08256820963993508.
[I 2023-03-21 15:38:49,866] Trial 1 finished with value: 0.21567392329663992 and parameters: {'n_estimators': 150, 'criterion': 'friedman_mse', 'min_samples_split': 2, 'bootstrap': True, 'max_samples': 0.16066807110964088, 'max_features': 1}. Best is trial 0 with value: 0.08256820963993508.
[I 2023-03-21 15:38:51,082] Trial 2 finished with value: 0.14419035653802095 and parameters: {'n_estimators': 10, 'criterion': 'friedman_mse', 'min_samples_split': 3, 'bootstrap': True, 'max_samples': 0.06841821675777426, 'max_features': 4}. Best is trial 0 with value: 0.08256820963993508.
[I 2023-03-21 15:39:33,463] Trial 3 finished 

model: RandomForrest test dataset error: 0.08256820963993508 best_params: {'bootstrap': True, 'criterion': 'friedman_mse', 'max_features': 4, 'max_samples': 0.16416466786708495, 'min_samples_split': 5, 'n_estimators': 110}


In [25]:
from sklearn.ensemble import ExtraTreesRegressor

def objective(trial):
    phase = min((trial.number // 30), 4)

    training_sizes = (0.01, 0.03, 0.1, 0.3, 1.0)
    training_max_duration_s = (120, 180, 360, 600, 1200)
    training_size = int(training_sizes[phase] * X_train_scaled.shape[0])
    max_duration_s = training_max_duration_s[phase]
    trial.set_user_attr("training_size", training_size)
    trial.set_user_attr("max_duration_s", max_duration_s)
    
    trial.set_user_attr("model_params", {
      "n_estimators": trial.suggest_int("n_estimators", 10, 200),
      "criterion": trial.suggest_categorical("criterion", ["squared_error", "absolute_error", "friedman_mse"]),
      "min_samples_split": trial.suggest_int("min_samples_split", 2, 5),
      "bootstrap": trial.suggest_categorical("bootstrap", [True]),
      "max_samples": trial.suggest_float("max_samples", 0.0, 0.2),
      "max_features": trial.suggest_int("max_features", 1, X_train_scaled.shape[1] // 2),
      "n_jobs": -1,
      "random_state": 42
    })
    
    model = ExtraTreesRegressor(**trial.user_attrs["model_params"])
    
    try:
        signal.alarm(max_duration_s)
        model.fit(X_train_scaled[:training_size,:], Y_train_pca[:training_size,:])
        signal.alarm(0)
        error = mse_on_test_dataset(model)
        save_best_model(model, error, trial)
    except KeyboardInterrupt as e:
        raise e
    except:
        raise optuna.exceptions.TrialPruned()
    return error

study = optuna.create_study(study_name="ExtraTreesRegressor", storage='sqlite:///../final/optuna.db', load_if_exists=True)
trials_due = 100 * RUN_ITERATIONS_PERCENT // 100 - len(study.trials)
if trials_due > 0:
    study.optimize(objective, n_trials=trials_due)
    
print(f"model: {study.study_name} test dataset error: {study.best_value} best_params: {study.best_params}")

[I 2023-03-21 16:06:15,586] A new study created in RDB with name: ExtraTreesRegressor


Updating ExtraTreesRegressor.pickle


[I 2023-03-21 16:06:17,671] Trial 0 finished with value: 0.1855282981273701 and parameters: {'n_estimators': 49, 'criterion': 'friedman_mse', 'min_samples_split': 4, 'bootstrap': True, 'max_samples': 0.18224843178696704, 'max_features': 2}. Best is trial 0 with value: 0.1855282981273701.


Updating ExtraTreesRegressor.pickle


[I 2023-03-21 16:07:08,445] Trial 1 finished with value: 0.10852403962715776 and parameters: {'n_estimators': 83, 'criterion': 'absolute_error', 'min_samples_split': 4, 'bootstrap': True, 'max_samples': 0.1343839267224416, 'max_features': 5}. Best is trial 1 with value: 0.10852403962715776.
[I 2023-03-21 16:07:10,827] Trial 2 finished with value: 0.1962629449947672 and parameters: {'n_estimators': 122, 'criterion': 'friedman_mse', 'min_samples_split': 3, 'bootstrap': True, 'max_samples': 0.13128280327545178, 'max_features': 2}. Best is trial 1 with value: 0.10852403962715776.
[I 2023-03-21 16:07:12,583] Trial 3 finished with value: 0.3824555826922991 and parameters: {'n_estimators': 66, 'criterion': 'friedman_mse', 'min_samples_split': 5, 'bootstrap': True, 'max_samples': 0.052000913115526085, 'max_features': 1}. Best is trial 1 with value: 0.10852403962715776.
[I 2023-03-21 16:07:14,935] Trial 4 finished with value: 0.17325219363605562 and parameters: {'n_estimators': 144, 'criterion'

model: ExtraTreesRegressor test dataset error: 0.10852403962715776 best_params: {'bootstrap': True, 'criterion': 'absolute_error', 'max_features': 5, 'max_samples': 0.1343839267224416, 'min_samples_split': 4, 'n_estimators': 83}


In [22]:
from sklearn.neighbors import KNeighborsRegressor

common_params = {
    "n_jobs": -1
}

def objective(trial):
    phase = min((trial.number // 30), 4)

    training_sizes = (0.01, 0.03, 0.1, 0.3, 1.0)
    training_max_duration_s = (120, 180, 360, 600, 1200)
    training_size = int(training_sizes[phase] * X_train_scaled.shape[0])
    max_duration_s = training_max_duration_s[phase]
    
    trial_params = {}
    trial_params["n_neighbors"] = trial.suggest_int("n_neighbors", 1, 100)
    trial_params["weights"] = trial.suggest_categorical("weights", ["uniform", "distance"])
    trial_params["algorithm"] = trial.suggest_categorical("algorithm", ["ball_tree", "kd_tree", "brute"])
    if trial_params["algorithm"] != "brute":
        trial_params["leaf_size"] = trial.suggest_int("leaf_size", 10, 50)
    trial_params["p"] = trial.suggest_int("p", 1, 5)
    
    model_params = {
        **trial_params,
        **common_params
    }
    trial.set_user_attr("model_params", model_params)
    model = KNeighborsRegressor(**trial.user_attrs["model_params"])
    
    try:
        signal.alarm(max_duration_s)
        model.fit(X_train_scaled[:training_size,:], Y_train_pca[:training_size,:])
        error = mse_on_test_dataset(model)
        signal.alarm(0)
    except KeyboardInterrupt as e:
        raise e
    except Exception as e:
        print(e)
        raise optuna.exceptions.TrialPruned()
    return error

study = optuna.create_study(study_name="KNeighborsRegressor", storage='sqlite:///../final/optuna.db', load_if_exists=True)
trials_due = 100 * RUN_ITERATIONS_PERCENT // 100 - len(study.trials)
if trials_due > 0:
    study.optimize(objective, n_trials=trials_due)
    
print(f"model: {study.study_name} test dataset error: {study.best_value} best_params: {study.best_params}")

[I 2023-03-21 15:43:48,684] A new study created in RDB with name: KNeighborsRegressor
[I 2023-03-21 15:44:06,034] Trial 0 finished with value: 0.18742825016871414 and parameters: {'n_neighbors': 26, 'weights': 'uniform', 'algorithm': 'brute', 'p': 3}. Best is trial 0 with value: 0.18742825016871414.
[I 2023-03-21 15:44:56,533] Trial 1 finished with value: 0.22331666645013348 and parameters: {'n_neighbors': 100, 'weights': 'uniform', 'algorithm': 'kd_tree', 'leaf_size': 42, 'p': 2}. Best is trial 0 with value: 0.18742825016871414.
[I 2023-03-21 15:46:15,761] Trial 2 finished with value: 0.20575789214344845 and parameters: {'n_neighbors': 53, 'weights': 'distance', 'algorithm': 'kd_tree', 'leaf_size': 19, 'p': 3}. Best is trial 0 with value: 0.18742825016871414.
[I 2023-03-21 15:47:13,638] Trial 3 finished with value: 0.1923036806813801 and parameters: {'n_neighbors': 70, 'weights': 'uniform', 'algorithm': 'ball_tree', 'leaf_size': 33, 'p': 1}. Best is trial 0 with value: 0.1874282501687

model: KNeighborsRegressor test dataset error: 0.15948482728059055 best_params: {'algorithm': 'kd_tree', 'leaf_size': 50, 'n_neighbors': 25, 'p': 1, 'weights': 'uniform'}
